In [ ]:
"""entonces es simplemente dado un estado y lo que se hizo en el estado para aumentar 
el score , luego es hacer factor comunt entre los estados y sus outputs para crear una
estructura equivalente.Pero entonces como hace para saber cual output seleccionar en
primer lieu ? c'est aleatoire s'il ne connait pas l'etat actuel ,apres tu pourra faire
la comparaison d'un nouvel etat par rapport a un autre connu en comparant le nombre de 
bits egaux de l'etat actuel"""

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [8]:
import pygame
import random

# Constants
GRID_SIZE = 20  # Size of each grid cell
MATRIX_WIDTH = 40  # Width in grid cells
MATRIX_HEIGHT = 15  # Height in grid cells (adjusted for the bottom section)
WIDTH, HEIGHT = 800, 600  # Total window dimensions
MATRIX_HEIGHT_PX = HEIGHT // 2  # Height for the matrix section (bottom half of the window)

# Initialize pygame
pygame.init()

# Screen setup
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Game and Matrix Window")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)

# Game objects
PLAYER_WIDTH, PLAYER_HEIGHT = 40, 40
FOOD_SIZE = 10
player1 = pygame.Rect(WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
player2 = pygame.Rect(3 * WIDTH // 4, HEIGHT // 4, PLAYER_WIDTH, PLAYER_HEIGHT)
food_list = [pygame.Rect(random.randint(0, WIDTH - FOOD_SIZE), random.randint(0, HEIGHT // 2 - FOOD_SIZE), FOOD_SIZE, FOOD_SIZE) for _ in range(5)]

# Fonts
font = pygame.font.Font(None, 36)

# Shared matrix for the game state
game_matrix = [[0] * MATRIX_WIDTH for _ in range(MATRIX_HEIGHT)]

# Game loop
clock = pygame.time.Clock()
running = True

def update_matrix():
    global game_matrix
    # Reset matrix
    for row in range(MATRIX_HEIGHT):
        for col in range(MATRIX_WIDTH):
            game_matrix[row][col] = 0

    # Update matrix with objects (players and food)
    # Player 1 (Block of 1s)
    for y in range(player1.top // GRID_SIZE, (player1.bottom) // GRID_SIZE):
        for x in range(player1.left // GRID_SIZE, (player1.right) // GRID_SIZE):
            if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                game_matrix[y][x] = 1
    
    # Player 2 (Block of 1s)
    for y in range(player2.top // GRID_SIZE, (player2.bottom) // GRID_SIZE):
        for x in range(player2.left // GRID_SIZE, (player2.right) // GRID_SIZE):
            if 0 <= y < MATRIX_HEIGHT and 0 <= x < MATRIX_WIDTH:
                game_matrix[y][x] = 1
    
    # Food (Block of 1s)
    for food in food_list:
        row = food.top // GRID_SIZE
        col = food.left // GRID_SIZE
        if 0 <= row < MATRIX_HEIGHT and 0 <= col < MATRIX_WIDTH:
            game_matrix[row][col] = 1

def draw_matrix():
    # Draw the matrix section (bottom part of the screen)
    matrix_surface = pygame.Surface((WIDTH, MATRIX_HEIGHT_PX))
    matrix_surface.fill(BLACK)

    font = pygame.font.Font(None, 24)
    for y, row in enumerate(game_matrix):
        row_text = " ".join(str(cell) for cell in row)
        text_surface = font.render(row_text, True, WHITE)
        matrix_surface.blit(text_surface, (10, y * 20))

    screen.blit(matrix_surface, (0, HEIGHT // 2))

def draw_game():
    # Draw the game section (top part of the screen)
    screen.fill(BLACK)
    pygame.draw.rect(screen, BLUE, player1)
    pygame.draw.rect(screen, RED, player2)
    for food in food_list:
        pygame.draw.rect(screen, GREEN, food)

def handle_movement():
    # Handle movement for player1 (WASD keys)
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w] and player1.top > 0:
        player1.y -= 5
    if keys[pygame.K_s] and player1.bottom < HEIGHT // 2:
        player1.y += 5
    if keys[pygame.K_a] and player1.left > 0:
        player1.x -= 5
    if keys[pygame.K_d] and player1.right < WIDTH:
        player1.x += 5

    # AI movement for player2
    if food_list:
        nearest_food = min(food_list, key=lambda f: abs(f.x - player2.x) + abs(f.y - player2.y))
        if nearest_food.x < player2.x:
            player2.x -= 5
        elif nearest_food.x > player2.x:
            player2.x += 5
        if nearest_food.y < player2.y:
            player2.y -= 5
        elif nearest_food.y > player2.y:
            player2.y += 5

def check_collisions():
    global food_list
    # Check for food collection by both players
    for food in food_list[:]:
        if player1.colliderect(food):
            food_list.remove(food)
        elif player2.colliderect(food):
            food_list.remove(food)

# Game loop
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    handle_movement()
    check_collisions()
    update_matrix()
    draw_game()
    draw_matrix()

    pygame.display.update()
    clock.tick(60)

pygame.quit()
